In [2]:
import os
os.chdir("../")

In [3]:
import glob
import pandas as pd
import time

In [18]:
resultDir = 'results'
problem = 'cauctions' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]
eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
eval_file = eval_files[-1]

In [31]:
df = pd.read_csv(eval_file)
df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

df = df[df['policy'] != 'internal:relpscost']

# 计算 one-shifted geometric means（未完成）
use uniform5 as basis, devide each instance's result by the corresponding one solved by uniform5. Then compute one-shifted geometric mean grouped by ['type','sampling_strategy']

In [22]:
# process_time = time.process_time()
# perf_time = time.perf_counter()
df2 = df[df['sampling_strategy'] == 'uniform5']
for i, row in df.iterrows():
    ref_row = df2[
        (df2['policy'] == df.loc[i].policy) &
        (df2['seed'] == df.loc[i].seed) &
        (df2['type'] == df.loc[i].type) &
        (df2['instance'] == df.loc[i].instance)
    ]
    df.at[i,'stime'] /= ref_row.stime
    df.at[i,'nnodes'] = df.at[i,'nnodes'].astype(float) / ref_row.nnodes.astype(float)
    df.at[i,'nlps'] = df.at[i,'nlps'].astype(float) / ref_row.nlps.astype(float)
    df.at[i,'walltime'] = df.at[i,'walltime'].astype(float) / ref_row.walltime.astype(float)
    df.at[i,'proctime'] = df.at[i,'proctime'].astype(float) / ref_row.proctime.astype(float)
 

In [23]:
import numpy as np

In [24]:
df.loc[(df['sampling_strategy'] == 'depthK') & (df['type']=='small'),'stime'].mean()

1.00176219724993

In [25]:
df.groupby(['type','sampling_strategy']).mean()

seed    nnodes      nlps     stime  gap  ndomchgs  \
type   sampling_strategy                                                      
medium depthK                2  0.994270  0.977001  0.979176  0.0         0   
       depthK2               2  1.010836  1.005307  1.005269  0.0         0   
       uniform5              2  1.000000  1.000000  1.000000  0.0         0   
small  depthK                2  1.064582  1.009630  1.001762  0.0         0   
       depthK2               2  0.996770  1.015081  0.984433  0.0         0   
       uniform5              2  1.000000  1.000000  1.000000  0.0         0   

                          ncutoffs  walltime  proctime  
type   sampling_strategy                                
medium depthK                    0  0.978819  0.979274  
       depthK2                   0  1.004671  1.005222  
       uniform5                  0  1.000000  1.000000  
small  depthK                    0  1.001426  1.002670  
       depthK2                   0  0.985637  0.986270  
       uniform5                  0  1.000000  1.000000

In [26]:
from scipy import stats

In [30]:
stats.mstats.gmean(df.loc[(df['sampling_strategy'] == 'depthK2') & (df['type']=='small'),'stime'] + 1) - 1

0.9801889882360184

# 计算 standard deviations
Average per-instance standard deviation (percentage) w.r.t different model training seeds

In [32]:
std_df = df.groupby(['type','sampling_strategy','instance']).std() / df.groupby(['type','sampling_strategy','instance']).mean()

In [33]:
std_df.groupby(['type','sampling_strategy']).mean()

seed    nnodes      nlps     stime  gap  \
type   sampling_strategy                                                
medium depthK             0.790569  0.119493  0.112478  0.085076  NaN   
       depthK2            0.790569  0.110848  0.112491  0.088538  NaN   
       uniform5           0.790569  0.121829  0.123748  0.102216  NaN   
small  depthK             0.790569  0.164647  0.161353  0.097555  NaN   
       depthK2            0.790569  0.147574  0.137920  0.087226  NaN   
       uniform5           0.790569  0.183908  0.157263  0.117446  NaN   

                          ndomchgs  ncutoffs  walltime  proctime  
type   sampling_strategy                                          
medium depthK                  NaN       NaN  0.085460  0.085714  
       depthK2                 NaN       NaN  0.089033  0.089232  
       uniform5                NaN       NaN  0.102078  0.102644  
small  depthK                  NaN       NaN  0.097720  0.098327  
       depthK2                 NaN       NaN  0.085903  0.086368  
       uniform5                NaN       NaN  0.119637  0.117503

# 计算 Wins

In [34]:
df_uniform5 = df[df['sampling_strategy']=='uniform5']
df_depthK = df[df['sampling_strategy']=='depthK']
df_depthK2 = df[df['sampling_strategy']=='depthK2']
df['Wins'] = 0

In [35]:
for i in range(0,len(df_uniform5)):
    uniform5_row = df_uniform5.iloc[i]
    depthK_row = df[
        (df['sampling_strategy'] == 'depthK') &
        (df['policy'] == uniform5_row.policy) &
        (df['seed'] == uniform5_row.seed) &
        (df['type'] == uniform5_row.type) &
        (df['instance'] == uniform5_row.instance)
    ]
    depthK2_row = df.loc[
        (df['sampling_strategy'] == 'depthK2') &
        (df['policy'] == uniform5_row.policy) &
        (df['seed'] == uniform5_row.seed) &
        (df['type'] == uniform5_row.type) &
        (df['instance'] == uniform5_row.instance)
    ]
    winner = np.argmin([uniform5_row.stime, depthK_row.stime.iloc[0], depthK2_row.stime.iloc[0]])
    winner = ['uniform5', 'depthK', 'depthK2'][winner]
    df.loc[ (df['sampling_strategy'] == winner) &
            (df['policy'] == uniform5_row.policy) &
            (df['seed'] == uniform5_row.seed) &
            (df['type'] == uniform5_row.type) &
            (df['instance'] == uniform5_row.instance), 'Wins'] += 1

In [36]:
df.groupby(['type', 'sampling_strategy']).sum()

seed    nnodes      nlps    stime  gap  ndomchgs  \
type   sampling_strategy                                                     
medium depthK              200  158347.0  195188.0  7196.85  0.0         0   
       depthK2             200  160996.0  201308.0  7277.01  0.0         0   
       uniform5            200  165408.0  206338.0  7428.09  0.0         0   
small  depthK              200   11016.0   22604.0   650.91  0.0         0   
       depthK2             200   10704.0   22847.0   641.19  0.0         0   
       uniform5            200   11160.0   23110.0   661.12  0.0         0   

                          ncutoffs     walltime     proctime  Wins  
type   sampling_strategy                                            
medium depthK                    0  7710.619490  7801.500000    41  
       depthK2                   0  7793.123748  7885.406250    26  
       uniform5                  0  7958.547362  8051.750000    33  
small  depthK                    0   704.797527   709.515625    25  
       depthK2                   0   695.123759   699.828125    46  
       uniform5                  0   716.004460   720.031250    29